In [1]:
from bt_data_feeds import PandasData
from Ornstein_Uhlenbeck import Ornstein_Uhlenbeck
import backtrader as bt
import numpy as np
import pandas as pd

In [2]:
stock_list = ['SLV', 'SIL']
stock_dict = {}
for stock in stock_list:
    stock_dict[stock] = pd.read_csv('data/{}.csv'.format(stock), index_col='Date', parse_dates=['Date'])

In [33]:
class MyStrategy(bt.Strategy):
    def __init__(self):
        self.St = {stock_list[0]: np.array([]),
                   stock_list[1]: np.array([])
                   }
        self.model = Ornstein_Uhlenbeck(stock_dict[stock_list[0]]['Adj Close'].to_numpy()[:252], 
                                        stock_dict[stock_list[1]]['Adj Close'].to_numpy()[:252])
        self.a = 0.01
        self.signal = 0
        self.pos = 0
        self.size = 5000000
    
    def signal_generation(self):
        Xt = self.model.alpha * self.St[stock_list[0]][-1] - self.model.beta * self.St[stock_list[1]][-1]
        
        if self.signal == 1:
            signal = 0 if Xt >= self.model.mu else 1
        elif self.signal == -1:
            signal = 0 if Xt <= self.model.mu else -1
        else:
            if Xt > np.quantile(self.model.X, 1 - self.a):
                signal = -1
            elif Xt < np.quantile(self.model.X, self.a):
                signal = 1
            else:
                signal = 0
        return signal
        
    def close_positions(self):
        for position in self.positions:
            self.close(data=position.data)
            
    def next(self):
        for data in self.datas:
            self.St[data._name] = np.append(self.St[data._name], data.adjclose[0])

        self.signal = self.signal_generation()
    
        if self.signal == 1:
            if self.pos == 0:
                self.buy(data=self.datas[0], size=self.size * self.model.alpha)
                self.sell(data=self.datas[1], size=self.size * self.model.beta)
        elif self.signal == -1:
            if self.pos == 0:
                self.sell(data=self.datas[0], size=self.size * self.model.alpha)
                self.buy(data=self.datas[1], size=self.size * self.model.beta)
        elif self.signal == 0:
            if self.pos != 0:
                self.close_positions()
                
        if len(self.St[stock_list[0]]) != 0 and len(self.St[stock_list[0]]) % 252 == 0:
            self.model = Ornstein_Uhlenbeck(self.St[stock_list[0]][-252:], self.St[stock_list[1]][-252:])

In [34]:
cerebro = bt.Cerebro()

data_feed_dict = {}
for stock in stock_list:
    data_feed_dict[stock] = PandasData(dataname=stock_dict[stock].iloc[252:], name=stock)
    cerebro.adddata(data_feed_dict[stock])

cerebro.addstrategy(MyStrategy)

# Run the backtest
cerebro.broker.setcash(100000000)
cerebro.run()
cerebro.broker.get_value()

195789899.88276768